Welcome to the second exercise of this summer school! During this exercise you will learn to apply different quality control procedures to (your) observational datasets. This exercise consists of three parts. Firstly, the necessary packages are again installed and imported, in the same way as the previous exercise. Secondly, the different functions from the toolkit concerning quality control are explained and applied to a demo dataset. Thirdly, you will apply the quality control capabilities from the toolkit to your own dataset.

# 1. Import and initialisation


## 1.1 Import the toolkit and additional packages

As each exercise is a separate Google Colab notebook, some initial steps from the previous exercise will have to be repeated in this new notebook. Firstly, the toolkit will have to be re-installed in the same way as before.

In [ ]:
!pip3 install git+https://github.com/vergauwenthomas/MetObs_toolkit
%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}

Next, we will again import the toolkit and other additional modules, which are necessary for this exercise.

In [ ]:
import metobs_toolkit
import pandas as pd
import datetime

Finally, you will also have to link your Google Drive to this notebook again.

In [ ]:
# Loading your Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


# 2. Quality control



## 2.1 Initial quality control

In this second part of the exercise we will work with a demo dataset. We will first load in this dataset in largely the same way as in the previous exercise, with one notable addition. Firstly, we create an empty dataset and we update the settings to link to the files from this demo dataset.

In [ ]:
# Make an empty dataset
dataset = metobs_toolkit.Dataset()

# Add the demo data files to the dataset settings
dataset.update_settings(input_data_file = metobs_toolkit.demo_datafile,
                        input_metadata_file = metobs_toolkit.demo_metadatafile,
                        data_template_file = metobs_toolkit.demo_template,
                        metadata_template_file = metobs_toolkit.demo_template # Contains also the metadata mapping
                        )

When you import the dataset some initial quality control steps are already exectuted:

1.   The toolkit looks for **duplicated timestamps**. As there is no way to know which of these timestamps are the correct ones, all of the duplicates are eliminated.
2.   **Invalid observations** are removed. For instance, when the dataset contains some text instead of a number, this is an invalid observation.
3.   Based on the observations, a time resolution is estimated for the dataset. With this time resolution, the toolkit searches for **missing observations**.
4.   When a series of consecutive missing observations is present, this is labeled as a **gap**. The minimum number of observations needed to define a gap is a number you can choose yourself, based on your preferences. This value can be modified in the quality control settings with the parameter ```gapsize_in_records```.

It is important to note that the toolkit looks for gaps at the moment the dataset is imported. Therefore, if you want to use a specific number of observations to define a gap, this needs to be defined before importing the data.

In this exercise we define a gap as a series of missing observations which lasts longer than 1 hour. As the time resolution of the demo dataset is 5 minutes, we hence set the parameter ```gapsize_in_records``` to 12, as there are 12 observations in 1 hour. We use the function ```update_qc_settings``` to perform this step:

In [ ]:
# Update the gap definition
dataset.update_qc_settings(gapsize_in_records=12)

Now we are ready to load in the dataset and take a look at what is inside:

In [ ]:
# Load the data from the demo data files
dataset.import_data_from_file()

In [ ]:
# Look at what is inside the dataset
dataset.show()

Observations that pass these intitial quality control steps are contained in the ```df``` object. Any observation that does not pass one of the initial quality control steps is not stored in this ```df```, but it is stored elsewhere depending on which check it failed.  labeled as an outlier. Observations with duplicated timestamps or invalid input are labeled as outliers and can be found in the ```outliersdf``` object:

In [ ]:
# The outliers are stored in the outliersdf object of the dataset:
outliers = dataset.outliersdf

# Print this object to see what is stored in this data frame:
print(outliers)

Each row denotes an observation that was flagged as an outlier. The observation is characterized by the name of the station, the timestamp and the observation type. Two columns then declare why this observation is an outlier. The first column ```"value"``` gives the value of this observation. If it is ```NaN```, then the observation is not a number. The second column ```"label"``` explains which quality control check this observation failed. Here, the label ```invalid input``` was given, because the observation was not numeric.

Missing observations are stored in the ```missing_obs``` object, while gaps are contained in a different object, aptly named ```gaps```. In the next exercise you will learn about gaps and how to handle them.

**For more information about the structure of a Dataset in this toolkit, you can consult the documentation [here](https://vergauwenthomas.github.io/MetObs_toolkit/intro.html#dataset).**

## 2.2 Additional quality control checks

Five additional quality control checks can be performed with the function ```apply_quality_control``` ([documentation](https://vergauwenthomas.github.io/MetObs_toolkit/_autosummary/metobs_toolkit.dataset.Dataset.html#metobs_toolkit.dataset.Dataset.apply_quality_control)). Each of the five available checks will be explained in the following exercises.

The quality control is performed based on settings that are associated with the dataset. The following code shows how these settings can be accessed. Take a quick look at the structure of this dictionary. It will become more clear when we will tackle the individual checks.





In [ ]:
# All settings, labels, replacement values are defined in the default settings in /settings_files/qc_settings.py
# To inspect (and change) these quality control settings, you can extract them:
qc_settings = dataset.settings.qc["qc_check_settings"]

# These settings are in a dictionary which contains multiple levels.
# The first level concerns the specific quality control check which the settings are for.
# You can print the keys of the dictionary to get an idea of the different available checks:
print(qc_settings.keys())

# All of these checks will be explained in the following parts of the exercise.

### 2.2.1 Gross value check

The [**gross value check**](https://vergauwenthomas.github.io/MetObs_toolkit/_autosummary/metobs_toolkit.qc_checks.gross_value_check.html#metobs_toolkit.qc_checks.gross_value_check) tests your dataset to see if the observations are between certain thresholds. The settings for this check can be found in the settings dictionary with the key ```"gross_value"```. For temperature these settings already have some default values. In the following code we will show how you can access these values.

In [ ]:
# The settings for the gross value check can be found in the qc_settings dictionary
# by using the key "gross_value"
print(qc_settings["gross_value"])

In [ ]:
# You can see that this dictionary only has one key: "temp".
# This is because default values are currently only given for temperature.
# The settings for temperature can be accessed in the following way:
print(qc_settings["gross_value"]["temp"])

These variables determine the minimum and maximum threshold used by the gross value check. What values would you choose for your own dataset?

**Extra information**: In this exercise we will only work with temperature. However, if you want to apply quality control to another variable, you can add this variable with the correct settings in the dictionary. For example, let us consider the relative humidity. This variable is expressed in % and needs to lie between 0 % and 100 %. The gross value check could then be applied to check this in the data.

### 2.2.2 Persistence check

The [**persistence check**](https://vergauwenthomas.github.io/MetObs_toolkit/_autosummary/metobs_toolkit.qc_checks.persistance_check.html#metobs_toolkit.qc_checks.persistance_check) searches for a consecutive series of repetitive observation values in your dataset. The check functions by looking at a certain time window to see if the observations are constant in this window. The length of this time window is defined in the settings by ``` "time_window_to_check" ```. This time windows should reflect a time interval during which you expect some variation in the observed variable. For the check to be executed, this time window should contain a minimum number of observations, which is determined by ``` "min_num_obs" ```. If all observations in the time window are identical, they are all labeled as a persistence outlier.

For temperature some default values are already set. Look for these values in the settings with the key ```"persistance"```. What values would you choose for your dataset?

In [ ]:
# Print the settings for the persistence check and find the default settings for the temperature

### 2.2.3 Repetitions check

The **[repetitions check](https://vergauwenthomas.github.io/MetObs_toolkit/_autosummary/metobs_toolkit.qc_checks.repetitions_check.html#metobs_toolkit.qc_checks.repetitions_check)** is very similar to the persistence check but works in a slightly different way. With the persistance check you define a certain time window during which you expect some variation in the observed variable. This time window is absent for the repetitions check: it simply checks the series of observations and looks for a series of consecutive constant values. A series of such constant values could indicate a connection error. In many cases the persistence check and the repetitions check will give the same results. However, in some cases, one of the checks will be more suitable, for example when the time resolution of your data is very coarse.

Can you find the default values for temperature (analogously to the two previous checks)?

In [ ]:
# Print the settings for the repetitions check and find the default settings for the temperature

### 2.2.4 Spike check

The **[spike check](https://vergauwenthomas.github.io/MetObs_toolkit/_autosummary/metobs_toolkit.qc_checks.step_check.html#metobs_toolkit.qc_checks.step_check)** inspects your dataset for abrupt changes in the observations between consecutive timestamps. If an observation varies too much from the previous observation, it is labeled as an outlier. You can find the settings for this check with the key ```"step"``` in the settings dictionary. The change between consecutive timestamps is expressed per second and is defined by the variables ```"max_increase_per_second"``` and ```"min_increase_per_second"``` in the settings.

Take a look at the default values for this check.

In [ ]:
# Print the settings for the spike check and find the default settings for the temperature

### 2.2.5 Window variation check

Lastly, while the spike check looks at the variation between two consecutive variations, the **[window variation check](https://vergauwenthomas.github.io/MetObs_toolkit/_autosummary/metobs_toolkit.qc_checks.window_variation_check.html#metobs_toolkit.qc_checks.window_variation_check)** investigates the variation of the data in a certain time window. This variation needs to be between a certain minimum and maximum threshold which can be found in the settings as respectively ```max_increase_per_second``` and ```min_increase_per_second```. The length of the time window is controlled by the variable ```time_window_to_check```, while ```min_window_members``` determines how many observations need to be inside of this window before performing this check. All of these settings can be found in the quality control settings dictionary under the key ```"window_variation"```.

Take a look at the default values for this check.

In [ ]:
# Print the settings for the window variation check and find the default settings for the temperature

## 2.3 Applying quality control

The five quality control checks can be applied to the dataset with the the function ```apply_quality_control``` ([documentation](https://vergauwenthomas.github.io/MetObs_toolkit/_autosummary/metobs_toolkit.dataset.Dataset.html#metobs_toolkit.dataset.Dataset.apply_quality_control)). Before applying the quality control, we will first coarsen the data to a time resolution of 1 hour.

In [ ]:
# Coarsen the time resolution to 1 hour
dataset.coarsen_time_resolution(freq='1H')

# Apply quality control
dataset.apply_quality_control(
    obstype="temp",         # choose which observations you want to check
    gross_value=True,       # set True if you want to perform the gross value check
    persistance=True,       # set True if you want to perform the persistence check
    repetitions=True,       # set True if you want to perform the repetitions check
    step=True,              # set True if you want to perform the spike check
    window_variation=True,  # set True if you want to perform the window variation check
)

This function updates the outliers data frame (```outliers_df```) in the dataset if some observations do not pass the previous quality control checks. Take a look at the outliers dataframe:

In [ ]:
# Print the outliers dataframe. Are there more outliers than before?
dataset.outliersdf.xs('temp', level='obstype') # Select only the temperature outliers

## 2.4 Plotting quality control results
You can plot the results of the full quality control nicely with the function below ([documentation](https://vergauwenthomas.github.io/MetObs_toolkit/_autosummary/metobs_toolkit.dataset.Dataset.html#metobs_toolkit.dataset.Dataset.get_qc_stats)). This function generates pie charts to display the quality control statistics. There is a general pie chart with the overall label of the observations: ok, outlier or missing. Next, there is also a general pie chart, specifying how the different types of outliers are distributed. Finally, each quality control check also has its own chart, denoting how many observations pass this check by labelling them as ok, outlier or not checked. Observations which are already labeled as an outlier are not checked again by the following checks, which results in the "not checked" label.

Inspect the code to generate the plot and look at the overview plot itself. Can you recognize all the features as decribed above?

In [ ]:
qc_statistics = dataset.get_qc_stats(
    obstype="temp",     # Specify which observation variable you want to get the statistics for; here we choose temperature
    stationname=None,   # None means all stations are plotted. You can also plot a specific station by specifying the station name, e.g. 'vlinder01'
    make_plot=True,     # Set True to make a plot
)

If you are interested in a specific station, you can do the quality control on a specific station rather than the full dataset. The following code gives an example of how this can be done. However, if you run this code right now, it will give an error because all quality control checks have already been performed for this dataset. If you want to do a new quality control procedure, you will have to start again from an empty dataset.

In [ ]:
specific_station = 'vlinder01' #the name of the station

station = dataset.get_station(specific_station)

station.apply_quality_control(
    obstype="temp",         # choose which observations you want to check
    gross_value=True,       # set True if you want to perform the gross value check
    persistance=True,       # set True if you want to perform the persistence check
    repetitions=True,       # set True if you want to perform the repetitions check
    step=True,              # set True if you want to perform the spike check
    window_variation=True,  # set True if you want to perform the window variation check
)

qc_statistics = station.get_qc_stats(
    obstype="temp",     # Specify which observation variable you want to get the statistics for; here we choose temperature
    make_plot=True,     # Set True to make a plot
)

When plotting a time series, the quality control outliers will also be present in the form of scatters on the time series. To visualise this use the `colorby='label'` attribute in the plotting function ([documentation](https://vergauwenthomas.github.io/MetObs_toolkit/_autosummary/metobs_toolkit.dataset.Dataset.html#metobs_toolkit.dataset.Dataset.make_plot)).

In [ ]:
dataset.make_plot(colorby="label")

You can also plot just the observations of one or more station of you choice. You can specify which station by using the ```stationnames``` argument of the plotting function:

In [ ]:
dataset.make_plot(colorby="label", stationnames=["vlinder05"]) # Here we plot only the observations of station 'vlinder05'

## 2.5 Changing the quality control settings

To change the settings used by the quality control you can use the [`update_qc_settings`](https://vergauwenthomas.github.io/MetObs_toolkit/_autosummary/metobs_toolkit.dataset_settings_updater.Dataset.html#metobs_toolkit.dataset_settings_updater.Dataset.update_qc_settings)function. You need to execute this function before applying the quality control.

In [ ]:
# Make an empty dataset
dataset = metobs_toolkit.Dataset()

# Add the demo data files to the dataset settings
dataset.update_settings(input_data_file = metobs_toolkit.demo_datafile,
                        input_metadata_file = metobs_toolkit.demo_metadatafile,
                        data_template_file = metobs_toolkit.demo_template,
                        metadata_template_file = metobs_toolkit.demo_template # Contains also the metadata mapping
                        )

# Update the settings
dataset.update_qc_settings(obstype='temp',
                           gross_value_max_value=27.2,
                           win_var_time_win_to_check='3H', # 3 hours
                           step_max_decrease_per_sec=3.6/3600,
                           gapsize_in_records=15)

# Load the data from the demo data files
dataset.import_data_from_file()

# Coarsen time resolution
dataset.coarsen_time_resolution(freq='1H')

# Apply quality control
dataset.apply_quality_control(obstype="temp")

# Visualise the effect
dataset.make_plot(obstype='temp', colorby='label')

# 3. Apply quality control to your our own dataset

After going through this exercise, you should now have a good idea of what quality control entails and how you can apply it using this toolkit. It is now up to you to use what you have learned and perform a quality control on your own dataset. Copy (parts of) the code from section 2 from this exercise to perform the following steps:

1.   Import your own dataset following the steps shown in the previous part of this exercise.
2.   Change the settings to improve the quality control for your data.
3.   Apply quality control to your own dataset
4.   Visualise the result

After performing these steps, you will save the quality controlled dataset for further use in the following exercises. This will be explained below.

**Step 1: Import your own dataset**

Copy (parts of) the code from section 2.1 to read in your own dataset. Do not forget to modify the length of a gap with ```gapsize_in_records``` in the quality control settings before importing the dataset. A recommended gap length is 3 hours. Remember that ```gapsize_in_records``` defines the length of a gap in terms of the number of missing observations, so you should define this value based on the time resolution of your dataset. If you are unsure what this time resolution is or if the time resolutions is variable, you can load in the dataset first and check the time resolution value in the ```metadf``` object under ```assumed_import_frequency```. Based on this value you can define an appropriate number for ```gapsize_in_records```.

In [ ]:
# Step 1: Import your own dataset

**Step 2: Update the QC settings**

Use the [`update_qc_settings`](https://vergauwenthomas.github.io/MetObs_toolkit/_autosummary/metobs_toolkit.dataset_settings_updater.Dataset.html#metobs_toolkit.dataset_settings_updater.Dataset.update_qc_settings) function (as in section 2.5) to update the QC settings of the different checks. Information about the checks can be found in section 2.2 of this exercise.

In [ ]:
# Step 2: Update the QC settings

**Step 3: Apply quality control**

Copy (parts of) the code from section 2.3 to apply quality control to your own dataset. For the following exercises it is important the dataset is coarsened to a time resolution of 1 hour! Make sure to **coarsen your dataset before applying quality control**.

In [ ]:
# Step 3: Apply quality control

**Step 4: Visualise the results**

Copy (parts of) the code from section 2.4 to visualise the results from the quality control. Use this step to verify if the quality controlled dataset matches your expectations. If not, you can repeat the steps above (starting from a new dataset in step 1) with some new settings until you acquire the desired result.

In [ ]:
# Step 4: Visualise the results

**Final step: save the quality controlled dataset**

As each exercise builds on the results from the previous exercises, it is important to save your dataset, so that you do not have to repeat all the previous steps when you continue working. Saving your dataset to a file can be easily done with the function ```save_dataset``` ([documentation](https://vergauwenthomas.github.io/MetObs_toolkit/_autosummary/metobs_toolkit.dataset.Dataset.html#metobs_toolkit.dataset.Dataset.save_dataset)). The dataset is save in a pickle file, with the extension ```.pkl```. In the next exercise you will import this dataset from this file and simply continue working where you left off.

In [ ]:
save_directory = # provide a directory where this dataset needs to be saved
filename = 'qc_controlled_dataset.pkl' # name of the file in which the dataset is saved
dataset.save_dataset(outputfolder = save_directory, filename=filename)

# 4. Extension

The quality control checks that are implemented in the toolkit are applied on each station, using only the observations of that station. Each of these checks looks for certain patterns in time to determine if observations pass the quality control check. If you are interested in using more advanced quality control, and if you have a dense network of observations, then **spatial quality control** checks can be applied.

Spatial quality control checks test the quality of observations by making use of observations at other locations. Sophisticated software exists that includes this type of quality control checks. An example of such software is [TITAN](https://asr.copernicus.org/articles/17/153/2020/).

It is possible in the MetObs-toolkit to apply one important spatial check from the TITAN framework to your Dataset: the [TITAN buddy check](https://vergauwenthomas.github.io/MetObs_toolkit/_autosummary/metobs_toolkit.dataset.Dataset.html#metobs_toolkit.dataset.Dataset.apply_titan_buddy_check).

Go through the documentation provided and try to apply the TITAN buddy check to your own dataset (or the demo dataset).